In [2]:
 from __future__ import print_function
import tensorflow as tf
import os
import import_ipynb
from skimage.transform import resize
from skimage.io import imsave
from data import load_test_data , load_train_data

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Dense , Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.activations import softmax
 
 
 
 
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
 
img_rows = 96
img_cols = 96
 
smooth = 1.



def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    
 
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
 
def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    print("++++++++++++++++++++++++++++++++++++++++++++++new Line")
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    print("dim",conv1.shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    print("dim",conv1.shape)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    print("dim",pool1.shape)
    
        
        
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    print('dim',conv2.shape)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    print('dim',conv2.shape)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    print('dim',pool2.shape)
    


    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    print('dim',conv3.shape)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    print('dim',conv3.shape)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    print('dim',pool3.shape)
    
 
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    print('dim',conv4.shape)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    print('dim',conv4.shape)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    print('dim',pool4.shape)
    
    
    
    print(tf.shape(conv4))
    td=conv4
    
    tx=softmax(td,axis=[1,2])
    q=1.31
    
    
    print("value of q",q)
    
    
    
    safe_x = K.maximum(tx,1e-6)
    new_p = K.switch(K.equal(q,1.), (K.log(safe_x)*safe_x)/K.log(2.718),(K.pow(safe_x,q))/(q-1))
    new_p = -new_p #-p^q/(q-1)
    
    if q!=1:
        new_p= 1+new_p
        
    print("new_p",new_p.shape)
    xyz=K.max(new_p,axis=[1,2],keepdims=True)
    print("xyz=",xyz.shape)
   
    
    print("xyz=",xyz.shape)
    weights=1-(new_p/xyz)
    print("weights",weights.shape)
    improved_p=td*weights
    print("improved_p=",improved_p.shape)
    improved_p=K.sum(improved_p,axis=[1,2])/144
    print("improved_pv2=",type(improved_p),improved_p.shape)
    
    ##################################
    
#     n=tx.shape[0]
#     temp=tf.zeros([0,0],tf.float32)
    
#     for i in range(n):
#         print("=========inside loop",i)
#         safe_x = K.maximum(tx[i],1e-6)
#         new_p = K.switch(K.equal(q,1.), K.log(safe_x),(K.pow(safe_x,q))/(q-1))
#         new_p = -new_p #-p^q/(q-1)
    
    
#         new_p= 1+new_p
#         weights=1-(new_p/K.max(new_p))
    
#         improved_p=td[i]*weights
        
#         temp=tf.concat([temp,(K.sum(improved_p)/144)],0)
    
    
    
    
#     dense_nodes=K.reshape(K.variable(temp),shape=(1,256))
#     print(dense_nodes.shape)
    ####
#     den=[]
#     for i in range(n):
#         den.append(tf.reduce_sum(before_gap[i])/144)
    
    
#    dense_nodes=tf.Variable(den)
#     dense_nodes=tf.reshape(dense_nodes,shape=(1,256))
    
#     dense_nodes=K.sum(before_gap,axis=1)/144.0
#     print(dense_nodes.shape)
#     dense_nodes=K.reshape(before_gap,shape=(1,256))
    
    
#     print("temp type=",type(temp[0]))
#     dense_nodes = tf. convert_to_tensor(improved_p)
#     print("dense node=",dense_nodes)
#     xy=tf.reshape(dense_nodes,(1,256))
#     print("xy=",xy)
    #improved_p=tf.reshape(improved_p,(1,256))
    
    #############################
    dense1=Dense(32,activation='relu')(improved_p)
    print("dense1",dense1)
    dense2=Dense(1,activation='softmax')(dense1)
    print("below dense",dense2)
    
    
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    print('dim',conv5.shape)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    print('dim',conv5.shape)
    
    
    print("conv5=",conv5)
 
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    print('dim',up6.shape)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    print('dim',conv6.shape)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    
    
 
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    print('dim',up7.shape)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    print('dim',conv7.shape)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    print('dim',conv7.shape)
    
 
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    print('dim',up8.shape)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    print('dim',conv8.shape)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    print('dim',conv8.shape)
   
 
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    print('dim',up9.shape)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    print('dim',conv9.shape)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    print('dim',conv9.shape)
    
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    print("dim", conv10.shape) 
 
    model = Model(inputs=[inputs], outputs=[conv10,dense2])
 
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=[dice_coef_loss,tf.keras.losses.BinaryCrossentropy()],
                  metrics=[dice_coef,tf.keras.metrics.BinaryCrossentropy()])
 
    return model
 
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
 
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p
 
def train_and_predict():
    imgs_train,imgs_mask_train,imgs_value_train=load_train_data()
 
    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)
 
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
 
    imgs_train -= mean
    imgs_train /= std
 
    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
 
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss' , mode = 'min' , verbose = 1 , save_best_only=True )
 
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, y=[imgs_mask_train,imgs_value_train], batch_size=32, epochs= 20, verbose=1, shuffle=True,
              validation_split=0.2 ,  callbacks=[model_checkpoint])
    
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    n_imgs_test = imgs_test.shape[0]
    #for i in range(n_imgs_test):

    #worked for 7 vs 1
    #imgs_mask_test, imgs_value_test= model.predict(np.expand_dims(imgs_test[0] , axis = 0), verbose=1)


    imgs_mask_test, imgs_value_test= model.predict(imgs_test, verbose=1)
    print("mask final img 1" , imgs_mask_test)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    
    # print(imgs_mask_test.shape , imgs_value_test.shape)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)


 
     
if __name__ == '__main__':
    train_and_predict()
        

------------------------------
Creating and compiling model...
------------------------------
++++++++++++++++++++++++++++++++++++++++++++++new Line
dim (None, 96, 96, 32)
dim (None, 96, 96, 32)
dim (None, 48, 48, 32)
dim (None, 48, 48, 64)
dim (None, 48, 48, 64)
dim (None, 24, 24, 64)
dim (None, 24, 24, 128)
dim (None, 24, 24, 128)
dim (None, 12, 12, 128)
dim (None, 12, 12, 256)
dim (None, 12, 12, 256)
dim (None, 6, 6, 256)
KerasTensor(type_spec=TensorSpec(shape=(4,), dtype=tf.int32, name=None), inferred_value=[None, 12, 12, 256], name='tf.compat.v1.shape_1/Shape:0', description="created by layer 'tf.compat.v1.shape_1'")
value of q 1.31
new_p (None, 12, 12, 256)
xyz= (None, 1, 1, 256)
xyz= (None, 1, 1, 256)
weights (None, 12, 12, 256)
improved_p= (None, 12, 12, 256)
improved_pv2= <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'> (None, 256)
dense1 KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='dense_2/Relu:0', description="create

   [2.49293066e-06]]]]
------------------------------
Saving predicted masks to files...
------------------------------


<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\100_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1000_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1004_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1006_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1008_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\101_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1115_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1118_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1123_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\113_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1135_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1136_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1247_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1248_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\125_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1250_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1251_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\1254_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07

...

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5341_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5342_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5347_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5348_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\535_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5354_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5477_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5478_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5479_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5480_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5481_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\5482_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac0

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\644_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\645_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\65_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\651_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\654_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\656_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286:

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\790_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\792_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\793_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\795_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\796_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\8_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: 

<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\929_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\932_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\934_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\936_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\937_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: UserWarning: preds\94_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-7c103ea7ac07>:286: